In [1]:
import os
import math
import time
import random
import itertools
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm.notebook import tqdm

In [2]:
# custom imports
import sys

sys.path.append(os.path.abspath(os.path.join('..', '..', 'utils')))

In [3]:
colors = [
    'blue', 'green', 'red', 'cyan', 'magenta',
    'yellow', 'black', 'orange', 'purple', 'brown',
    'pink', 'grey', 'maroon', 'gold', 'chocolate',
    'aqua', 'darkviolet', 'crimson', 'navy', 'darkgreen',
    'peru', 'tan', 'seagreen', 'darkslategrey', 'teal'
]

In [4]:
# constants
A = 3e-4
use_precise_transform = False

## Let's load the data files

In [5]:
data_dir = os.path.join('..', '..', '..', 'data')

root_dirs = {
    'ideal':
        os.path.join(data_dir, 'pdg13-n25-0.5to10GeV-0.5eta'),
    'mat':
        os.path.join(data_dir, 'pdg13-n25-0.5to10GeV-0.5eta-with-material-effects'),
    'odd-bfield':
        os.path.join(data_dir, 'pdg13-n25-0.5to10GeV-0.5eta-non-homogenous-magnetic-field'),
    'mat-odd-bfield':
        os.path.join(data_dir, 'pdg13-n25-0.5to10GeV-0.5eta-with-material-effects-non-homogenous-magnetic-field')
}


# read the hits files and dataset
hit_files = {
    _type: sorted([file for file in os.listdir(root_dir) if file.endswith("-hits.csv")])
    for _type, root_dir in root_dirs.items()
}
hit_dfs = {
    _type: [pd.read_csv(os.path.join(root_dirs[_type], file), dtype={'particle_id':str, 'geometry_id': str})
            for file in files]
    for _type, files in hit_files.items()
}

# read the initial files and dataset
initial_files = {
    _type: sorted([file for file in os.listdir(root_dir) if file.endswith("-particles_initial.csv")])
    for _type, root_dir in root_dirs.items()
}
initial_dfs = {
    _type: [pd.read_csv(os.path.join(root_dirs[_type], file), dtype={'particle_id':str, 'geometry_id': str})
            for file in files]
    for _type, files in initial_files.items()
}

# read the final files and dataset
final_files = {
    _type: sorted([file for file in os.listdir(root_dir) if file.endswith("-particles_final.csv")])
    for _type, root_dir in root_dirs.items()
}
final_dfs = {
    _type: [pd.read_csv(os.path.join(root_dirs[_type], file), dtype={'particle_id':str, 'geometry_id': str})
            for file in files]
    for _type, files in final_files.items()
}

## Let's pick an event at randrom for every case and plot the x-y and r-x views

In [ ]:
# num_events = 100
# random.seed(682021)

# # pick an event randomly for each case
# random_event = random.choice(range(0, num_events))
# ideal_df = hit_dfs['ideal'][random_event]
# ideal_df['r'] = np.sqrt(np.square(ideal_df['tx']) + np.square(ideal_df['ty']))
# ideal_df['phi'] = np.arctan2(ideal_df['ty'], ideal_df['tx'])
# ideal_df['xy_track'] = ideal_df[['r','phi']].apply(lambda pair: (pair[0], pair[1]), 1)
# ideal_df['rz_track'] = ideal_df[['r','tz']].apply(lambda pair: (-pair[0], pair[1]), 1)
# print(f"Event chosen for the ideal simulation:\t\t\t\t\t\t{hit_files['ideal'][random_event]}")

# random_event = random.choice(range(0, num_events))
# mat_df = hit_dfs['mat'][random_event]
# mat_df['r'] = np.sqrt(np.square(mat_df['tx']) + np.square(mat_df['ty']))
# mat_df['phi'] = np.arctan2(mat_df['ty'], mat_df['tx'])
# mat_df['xy_track'] = mat_df[['r','phi']].apply(lambda pair: (pair[0], pair[1]), 1)
# mat_df['rz_track'] = mat_df[['r','tz']].apply(lambda pair: (-pair[0], pair[1]), 1)
# print(f"Event chosen for the simulation with material effect:\t\t\t\t{hit_files['mat'][random_event]}")

# random_event = random.choice(range(0, num_events))
# odd_bfield_df = hit_dfs['odd-bfield'][random_event]
# odd_bfield_df['r'] = np.sqrt(np.square(odd_bfield_df['tx']) + np.square(odd_bfield_df['ty']))
# odd_bfield_df['phi'] = np.arctan2(odd_bfield_df['ty'], odd_bfield_df['tx'])
# odd_bfield_df['xy_track'] = odd_bfield_df[['r','phi']].apply(lambda pair: (pair[0], pair[1]), 1)
# odd_bfield_df['rz_track'] = odd_bfield_df[['r','tz']].apply(lambda pair: (-pair[0], pair[1]), 1)
# print(f"Event chosen for the simulation with non-homogenous B:\t\t\t\t"
#       f"{hit_files['odd-bfield'][random_event]}")

# random_event = random.choice(range(0, num_events))
# mat_odd_bfield_df = hit_dfs['mat-odd-bfield'][random_event]
# mat_odd_bfield_df['r'] = np.sqrt(np.square(mat_odd_bfield_df['tx']) + np.square(mat_odd_bfield_df['ty']))
# mat_odd_bfield_df['phi'] = np.arctan2(mat_odd_bfield_df['ty'], mat_odd_bfield_df['tx'])
# mat_odd_bfield_df['xy_track'] = mat_odd_bfield_df[['r','phi']].apply(lambda pair: (pair[0], pair[1]), 1)
# mat_odd_bfield_df['rz_track'] = mat_odd_bfield_df[['r','tz']].apply(lambda pair: (-pair[0], pair[1]), 1)
# print(f"Event chosen for the simulation with material effect and non-homogenous B:\t"
#       f"{hit_files['mat-odd-bfield'][random_event]}")

In [ ]:
# from notebook1_utils import plot_views

# dfs = [ideal_df, mat_df, odd_bfield_df, mat_odd_bfield_df]
# types = ['"ideal"',
#          '"with-material-effects"',
#          '\n"non-homogenous-magnetic-field"',
#          '\n"with-material-effects-and-non-homogenous-magnetic-field"'
#         ]

In [ ]:
# plot_views(dfs, types, 'tx', 'ty', 'x', 'y', colors)

In [ ]:
# plot_views(dfs, types, 'tz', 'r', 'z', 'r', colors)

The distortion from the material effect and the non-homogenous magnetic field is not all that visible in these plots (at least to my eye). Let's take a look at the Hough Space also.

In [ ]:
# from notebook1_utils import plot_xy_hough

# xrange = (-np.pi, np.pi)
# ylims = (-50, 50)
# plot_xy_hough(dfs, A, types, xrange, ylims)

In [ ]:
# from notebook1_utils import plot_rz_hough

# xrange = (-1.5, 1.5)
# ylims = (-25, 25)
# plot_rz_hough(dfs, types, xrange, ylims)

The `x_range` and `ylims` parameters above can be configured in order to zoom in on any part of the plots, in order to inspect it further. Now, we will apply the Hough Transforms over the whole datasets to see how the introduction of material and non-homogenous magnetic field will assess the performance of the algorothm.

# Let's apply the Hough Transform to the whole datasets

In [6]:
transforms = ['xy', 'rz', 'xy-rz-combo', 'rz-xy-combo']
evaluation_metrics = ['efficiency', 'fake', 'duplicate']
simulations = list(root_dirs.keys())

stats = {
    metric: {transform: {_type: 0 for _type in simulations} for transform in transforms}
    for metric in evaluation_metrics
}
stats

{'efficiency': {'xy': {'ideal': 0,
   'mat': 0,
   'odd-bfield': 0,
   'mat-odd-bfield': 0},
  'rz': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'xy-rz-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'rz-xy-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0}},
 'fake': {'xy': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'rz': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'xy-rz-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'rz-xy-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0}},
 'duplicate': {'xy': {'ideal': 0,
   'mat': 0,
   'odd-bfield': 0,
   'mat-odd-bfield': 0},
  'rz': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'xy-rz-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0},
  'rz-xy-combo': {'ideal': 0, 'mat': 0, 'odd-bfield': 0, 'mat-odd-bfield': 0}}}

In [7]:
# define the hyperparameters of the transforms
xy_hyperparams = {
    'bin-size': (0.005, 0.1),
    'xrange': (-np.pi, np.pi),
    'yrange': (-1000, 1000),
    'minimum-hits-per-bin': 10,
    'A': A,
    'use-precise-transform': False
}

rz_hyperparams = {
    'bin-size': (0.001, 0.05),
    'xrange': (-2, 2),
    'yrange': (-5000, 5000),
    'minimum-hits-per-bin': 12
}

In [ ]:
from notebook1_utils import run_pipeline_over_whole_datasets

run_pipeline_over_whole_datasets(hit_dfs, stats, xy_hyperparams, rz_hyperparams)

In [24]:
rz_hyperparams = {
    'bin-size': (0.001, 0.01),
    'xrange': (-1, 1),
    'yrange': (-2000, 2000),
    'minimum-hits-per-bin': 10
}

In [28]:
# from notebook1_utils import Hough2d_pipeline, compute_qpt, compute_b
# from metrics import efficiency_rate, fake_rate, duplicate_rate

# sum_efficiency_rates = {'ideal': 0}
# sum_fake_rates = {'ideal': 0}
# sum_duplicate_rates = {'ideal': 0}

# ideal_dfs = hit_dfs['ideal']
# for df in tqdm(ideal_dfs):

#     df['weight'] = 1.0
#     df['r'] = np.sqrt(np.square(df['tx']) + np.square(df['ty']))
#     df['phi'] = np.arctan2(df['ty'], df['tx'])
#     df['xy_track'] = df[['r','phi']].apply(lambda pair: (pair[0], pair[1]), 1)
#     df['rz_track'] = df[['r','tz']].apply(lambda pair: (-pair[0], pair[1]), 1)
    
#     tracks = list(df['rz_track'])
#     _, rz_est = Hough2d_pipeline(tracks, rz_hyperparams, compute_b)
        
#     df['track'] = df['rz_track']
#     sum_efficiency_rates['ideal'] += efficiency_rate(rz_est.values(), df)
#     sum_fake_rates['ideal'] += fake_rate(rz_est.values(), df)
#     sum_duplicate_rates['ideal'] += duplicate_rate(rz_est.values(), df)
    
# sum_efficiency_rates = {key: val / 100 for key, val in sum_efficiency_rates.items()}
# sum_efficiency_rates

In [29]:
# sum_duplicate_rates = {key: val / 100 for key, val in sum_duplicate_rates.items()}
# sum_duplicate_rates

In [30]:
# sum_fake_rates = {key: val / 100 for key, val in sum_fake_rates.items()}
# sum_fake_rates

### First we will run each transformation to see how it performs on its own and then we will compare them with their combination.

In [ ]:
# # first define the hyperparameters of the transforms
# xy_hyperparams = {
#     'bin-size': (0.0025, 0.05),
#     'phi-range': (0, np.pi),
#     'qpt-range': (-200, 200),
#     'minimum-hits-per-bin': 10,
#     'use-sin': True
# }

# rz_hyperparams = {
#     'bin-size': (0.005, 0.1),
#     'r-range': (-1, 1),
#     'z-range': (-1000, 1000),
#     'minimum-hits-per-bin': 12
# }

In [ ]:
# from notebook2_utils import xy_pipeline, compute_all_intersections, rz_pipeline

# _, xy_est_tracks_to_hits = xy_pipeline(all_xy_tracks, xy_hyperparams)

# rz_intersections = compute_all_intersections(all_rz_tracks)
# rz_est_tracks_to_hits = rz_pipeline(all_rz_tracks, rz_intersections, rz_hyperparams)

# len(xy_est_tracks_to_hits), len(rz_est_tracks_to_hits)

In [ ]:
# from metrics import efficiency_rate, fake_rate, duplicate_rate

# df['track'] = df['xy_track']
# xy_eff = efficiency_rate(xy_est_tracks_to_hits.values(), df)
# xy_fak = fake_rate(xy_est_tracks_to_hits.values(), df)
# xy_dup = duplicate_rate(xy_est_tracks_to_hits.values(), df)

# df['track'] = df['rz_track']
# rz_eff = efficiency_rate(rz_est_tracks_to_hits.values(), df)
# rz_fak = fake_rate(rz_est_tracks_to_hits.values(), df)
# rz_dup = duplicate_rate(rz_est_tracks_to_hits.values(), df)